In [3]:
from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime
from datetime import datetime, timedelta
import urlparse
import pymongo 
from pymongo import MongoClient
import os

In [4]:
connection = MongoClient()
db = connection['espn_leagues_drafts']
snake_drafts = db['snake_drafts']

In [5]:
def get_league_draft_picks(soup):
     
    league_picks = []

    draft = soup.find("div", {"class": "games-fullcol games-fullcol-extramargin"})
    
    if draft == None:
        return None
        
    draft_deets = draft.find("div", {"class": "games-alert-mod alert-mod2 games-grey-alert"})
    for deet in draft_deets.findAll('b'):
        if str(deet.text).strip() == "Type:":
            draft_type=deet.nextSibling.strip()
            if draft_type != "Snake Draft":
                return None
    draft_results=draft.find('table')

    for round_num, draft_round in enumerate(draft_results.findAll('table')):
        
        for pick in draft_round.findAll('tr',{"class":"tableBody"}):
            pick_dict = {}
            for i, td in enumerate(pick.findAll('td')):
                if i == 0:
                    pick_num= int(td.text)
                elif i == 1: 
                    player_link = td.find('a')
                    if not player_link == None:
                        player_name = player_link.text
                        player_id = player_link['playerid']
                        unique_team_id = player_link['teamid']
                    else:
                        player_name = ''
                        player_id = -9999999999
                        unique_team_id = -9999999999
                elif i == 2:
                    team = td.find('a')
                    team_link = team['href']
                    team_name=team.text
                    parsed=urlparse.urlparse(team_link)
                    team_id=urlparse.parse_qs(parsed.query)['teamId'][0]
                    season_id = urlparse.parse_qs(parsed.query)['seasonId'][0]
                    league_id = urlparse.parse_qs(parsed.query)['leagueId'][0]
                    

                else:
                    raise

            pick_dict = {"league_id":league_id,
                        "team_name":team_name,
                        "team_id":team_id,
                        "unique_team_id":unique_team_id,
                        "season_id":season_id,
                        "player_name":player_name,
                        "player_id":player_id,
                        "pick_num":int(pick_num),
                        "round_num":round_num+1}

            league_picks.append(pick_dict)
    
    snake_drafts.insert_many(league_picks)
    
    return None

In [12]:
def get_all_picks(year):

    all_picks = []

    for draft_recap in os.listdir(year):
        if not draft_recap.startswith("."):
            
            draft_file = open('%s/%s'%(year,draft_recap),'r')
            soup = BeautifulSoup(draft_file,"lxml")

            try:
                get_league_draft_picks(soup)
                
            except:
                print draft_recap, "failed"
            
            draft_file.close()
            
    return all_picks

In [13]:
get_all_picks('2015')

draftrecap?leagueId=26906 failed


[]